## Intro

- Sofia Heisler의 A Beginner’s Guide to Optimizing Pandas Code for Speed 의 번역을 참조하였다.
- 초창기에 판다스는 느리다는 평가를 받았다고 한다.
- 물론 최적화가 되어있는 c 코드를 이길수는 없을것이지만, 어느정도 '잘' 짜여진 판다스 코드는 충분히 빠르다.
- 하지만 느린 코드를 짜고 파이썬도 느리구나.... 라는 불평을 하는 사람은 마땅히 벌을 받아야 할 것이다. 
    - 내가 주로 다루는 데이터가 1기가가 넘는 데이터가 거의 없었으니, 그냥 for 문으로 돌려놓고 유튜브 한번 보고 오면 다 돌아가는 수준이였다. 
    - 그래서 최적화에 관심을 가지지 않았는데, 최근들어 EDA 와 동시에 작업해야되는 일이 많아지면서 (시각화, 분석 등...) 점점 느려터진 속도에 화가 나던 중, 문득 최적화에 대해서는 찾아본적이 없다는것을 깨닫고 찾아본 결과 내 코드는 쓰레기 그 자체였다.
    - 앞으로는 올바른 판다스 문법으로 안좋은 코드를 모두 고칠 계획이다.(컴퓨터야 미안해!)

In [129]:
import pandas as pd
import numpy as np

In [22]:
from seaborn import load_dataset
diamonds=load_dataset('diamonds')
df = diamonds.copy()
df.head()

,carat,cut,color,clarity,depth,table,price,x,y,z
0,0.23,Ideal,E,SI2,61.5,55.0,326,3.95,3.98,2.43
1,0.21,Premium,E,SI1,59.8,61.0,326,3.89,3.84,2.31
2,0.23,Good,E,VS1,56.9,65.0,327,4.05,4.07,2.31
3,0.29,Premium,I,VS2,62.4,58.0,334,4.20,4.23,2.63
4,0.31,Good,J,SI2,63.3,58.0,335,4.34,4.35,2.75


In [4]:
df.describe()

,carat,depth,table,price,x,y,z
count,53940.000000,53940.000000,53940.000000,53940.000000,53940.000000,53940.000000,53940.000000
mean,0.797940,61.749405,57.457184,3932.799722,5.731157,5.734526,3.538734
std,0.474011,1.432621,2.234491,3989.439738,1.121761,1.142135,0.705699
min,0.200000,43.000000,43.000000,326.000000,0.000000,0.000000,0.000000
25%,0.400000,61.000000,56.000000,950.000000,4.710000,4.720000,2.910000
50%,0.700000,61.800000,57.000000,2401.000000,5.700000,5.710000,3.530000
75%,1.040000,62.500000,59.000000,5324.250000,6.540000,6.540000,4.040000
max,5.010000,79.000000,95.000000,18823.000000,10.740000,58.900000,31.800000


## for 문

- 내가 했던 실수들의 예시이다! 
- 데이터 프레임 행을 하나씩 반복하여서, 적용하려 한다.
- 이 방법의 경우 매우 직관적이다. 우리가 문법을 배울때도, for , while 등의 구문을 먼저 배우며 또한 이런 구조를 index(데이터) 각각에 적용하는것도 매우 직관적.
- 하지만 최적화 면에서 매우 구리다.

In [78]:
# 캐럿수가 크고, 가격도 크면 좋은 다이아몬드라고 1을 주고 싶다.
def cal(df):
    for i in range(0,len(df)):
        if (df.loc[i,'carat'] > 0.4) & (df.loc[i,'price'] > 300) :
            df.loc[i,'good'] = 1

In [79]:
%%timeit -n 1 -r 1 # 1번 roop, 1개의 best : 즉 그냥 한번 돌린 시간을 재겠다는 것이다.
cal(df)
# 실행하는데에 시간이 좀 걸리는 모습을 볼 수 있다.
# 이정도 속도면, data columns 이 100만개정도 되면 거의 2~3분 걸리는거다!

13.5 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [80]:
sum(df['good'])

39549

## Iterrows 방법

- iterrows 메서드는 데이터 프레임의 행을 반복하며 행행 자체를 포함하는 객체+index 를 반환해준다.
- for 문과 뭐가 다르는지 궁금하겠지만, iterrows 는 pandas 와 잘 작동하도록 최적화가 되어있다. 
- 같은 문제를 평균적으로 for 문 보다 4배이상 빠르게 해결한다고 한다.

In [69]:
df = diamonds.copy()

In [70]:
df.iloc[[1]]

,carat,cut,color,clarity,depth,table,price,x,y,z
1,0.21,Premium,E,SI1,59.8,61.0,326,3.89,3.84,2.31


- 먼저 idx 함수를 정의해서 index 와 row를 어떻게 뽑아내는지 알아보자.

In [71]:
def idx(df):
    for index,row in df.iloc[[0]].iterrows() : # itterrows 를 적용한 뒤에 index 와 row 를 적용
        return index,row

In [72]:
index,row=idx(df)

- 아래와 같이 index 는 수, row 는 pandas series 의 형태로 나오게 된다.

In [73]:
print(type(row))
print(index)

<class 'pandas.core.series.Series'>
0


In [56]:
for index,row in df.head().iterrows():
    print(index)
    print(row)

0
carat       0.23
cut        Ideal
color          E
clarity      SI2
depth       61.5
table         55
price        326
x           3.95
y           3.98
z           2.43
Name: 0, dtype: object
1
carat         0.21
cut        Premium
color            E
clarity        SI1
depth         59.8
table           61
price          326
x             3.89
y             3.84
z             2.31
Name: 1, dtype: object
2
carat      0.23
cut        Good
color         E
clarity     VS1
depth      56.9
table        65
price       327
x          4.05
y          4.07
z          2.31
Name: 2, dtype: object
3
carat         0.29
cut        Premium
color            I
clarity        VS2
depth         62.4
table           58
price          334
x              4.2
y             4.23
z             2.63
Name: 3, dtype: object
4
carat      0.31
cut        Good
color         J
clarity     SI2
depth      63.3
table        58
price       335
x          4.34
y          4.35
z          2.75
Name: 4, dtype: object


- 이제 위의 iterrows 를 가지고 for 문으로 하려 했던, 0.4 캐럿, 300 이상의 금액일 때에 새로운 good 열에 1 을 할당하는 함수를 돌려보자.

In [82]:
%%timeit -n 1 -r 1
good_list = []
for index,row in df.iterrows():
    if (row['carat'] > 0.4)&(row['price']>300):
        good_list.append(1) # list 에 저장해야된다.
    else :
        good_list.append(0)
df['good'] = good_list

3.56 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [83]:
sum(df['good']) # 맞게변환된것을 볼 수 있다.

39549

## 벡터화

- 판다스의 기본 단위인 데이터 프레임과 시리즈는 모두 배열 기반이다.
- 벡터화 한 다음 계산하게 되면 엄청나게 빠르다.

In [125]:
%%timeit -n 100 -r 3
df.loc[(df['carat'] > 0.4)&(df['price']>300),'good'] = 1

1.21 ms ± 83.4 µs per loop (mean ± std. dev. of 3 runs, 100 loops each)


In [112]:
sum(df['good']) 

39549

## 넘파이로 바꿔서 연산

- 넘파이는 '과학 계산을 위한 파이썬 기본 패키지' 를 표방한다.
- 즉 내부가 최적화된 사전 컴파일 된 C 코드로 작업을 수행하게 된다.
- 판다스와 마찬가지로 넘파이는 배열객체에서 작동하지만 INDEX, 데이터 유형 확인 등 판다스 시리즈 작업시 필요한 불필요한 작업을 피할 수 있어 매우매우 빠르다.

- 만약 index를 사용해야 하는 연산이면 벡터화만으로 충분하지만, index 를 쓸 필요가 없고 매우 빠른 연산을 하고 싶다면 value 메서드를 이용해서 배열화 하는것이 빠르게 할 수 있을것이다.

In [128]:
%%timeit -n 100 -r 3
df['good'] = ((df['carat'].values > 0.4)&(df['price'].values>300))*1

277 µs ± 33.6 µs per loop (mean ± std. dev. of 3 runs, 100 loops each)
